## Create netCDF for complex DSGs

The CF Conventions provided recommendations on netCDF structure for two types of complex DSGs, i.e. **time series of profiles** and **trajectory of profiles**.


### Time Series of Profiles

* Time series of profiles at a single station [(H.5.2)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_time_series_of_profiles_at_a_single_station): There is only one stations in a dataset; for this station, multiple profiles were measured at multiple times.

* Multidimensional array representations of time series profiles [(H.5.1)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_multidimensional_array_representations_of_time_series_profiles): There are multiple stations in a dataset; for each station, one or more profiles were measured at multiple times; and there are the same number of profiles for each station, and the same number of vertical levels for every profile, whereas the time values and the vertical level values can, but don't have to be the same. *For the situation where all time series share common time values, and all profiles share common vertical level values*, we can use the representation in the example H.17 of Appendix H.5.1 for a more efficient use of storage space.

* Ragged array representation of time series profiles [(H.5.3)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_ragged_array_representation_of_time_series_profiles): There are multiple stations, the number of profiles and vertical levels for each station varies.


### Trajectory of Profiles

* Profiles along a single trajectory [(H.6.2)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_profiles_along_a_single_trajectory)

* Multidimensional array representation of trajectory profiles [(H.6.1)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_multidimensional_array_representation_of_trajectory_profiles): There're multiple trajectories in the file, and there're the same number of profiles for all trajectories and the same number of vertical levels for all profiles. This representation can also be used for trajectories with different number of profiles or vertical levels, at the cost of some wasted storage space due to missing values in the arrays.

* Ragged array representation of trajectory profiles [(H.6.3)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_ragged_array_representation_of_trajectory_profiles): The number of profiles and the vertical levels for each trajectory varies.

```{note}
It is not always a good practice to pack all data into one single netCDF file. If the data structure becomes too complicated, it is often a better option to separate the features e.g. by stations and put them into different netCDF files.
```

For example, the [Global Temperature and Salinity Profile Programme (GTSPP)](https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdGtsppBest.html) dataset from NOAA is a ragged array representation of trajectory of profiles trajectories of profiles (H.6.3). &darr;

In [17]:
import os
from glob import glob
import xarray as xr

# List available datasets. Please change it to your file path.
os.chdir("/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data")
trjP_files = sorted(glob(os.path.join(os.getcwd(), "dsg_trjProfile", "*.nc")))
ds_trjP = xr.open_dataset(trjP_files[1])
ds_trjP.info()

xarray.Dataset {
dimensions:
	trajectory = 371 ;
	profile = 132501 ;
	obs = 419569 ;

variables:
	object trajectory(trajectory) ;
		trajectory:cf_role = trajectory_id ;
		trajectory:comment = Constructed from org_type_platform_cruise ;
		trajectory:ioos_category = Identifier ;
		trajectory:long_name = Trajectory ID ;
	object org(trajectory) ;
		org:comment = From the first 2 characters of stream_ident:
Code  Meaning
AD  Australian Oceanographic Data Centre
AF  Argentina Fisheries (Fisheries Research and Development National Institute (INIDEP), Mar del Plata, Argentina
AO  Atlantic Oceanographic and Meteorological Lab
AP  Asia-Pacific (International Pacific Research Center/ Asia-Pacific Data-Research Center)
BI  BIO Bedford institute of Oceanography
CF  Canadian Navy
CS  CSIRO in Australia
DA  Dalhousie University
FN  FNOC in Monterey, California
FR  Orstom, Brest
FW  Fresh Water Institute (Winnipeg)
GE  BSH, Germany
IC  ICES
II  IIP
IK  Institut fur Meereskunde, Kiel
IM  IML
IO  IOS in

### Exercise: create a netCDF for time series of profiles

Now, let's practice creating a netCDF for time series of profiles with the [Newport Lab CTD Casts](https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdNewportCtd.html) dataset. Each original file contains a single profile at a time, with measurements of sea water temperature, salinity, density, and fluorescence in multiple water depths. Since each profile has different number of water depth levels, we'll compose the netCDF in a ragged array representation of time series profiles (H.5.3). You could randomly [download](https://coastwatch.pfeg.noaa.gov/erddap/files/erdNewportCtd/) a few original files to do this exercise; if you encounter any difficulties, feel free to have a look at the provided sample solution. The sample solution used three datasets, [010903NH05.nc](https://coastwatch.pfeg.noaa.gov/erddap/files/erdNewportCtd/NH05/), [011207NH05.nc](https://coastwatch.pfeg.noaa.gov/erddap/files/erdNewportCtd/NH05/) and [061300NH2.nc](https://coastwatch.pfeg.noaa.gov/erddap/files/erdNewportCtd/NH02/).

In [18]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import cftime
from datetime import datetime

In [83]:
# List the downloaded original files
tsP_files = sorted(glob(os.path.join(os.getcwd(), "dsg_tsProfile", "*.nc")))
tsP_files

['/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_tsProfile/010903NH05.nc',
 '/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_tsProfile/011207NH05.nc',
 '/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_tsProfile/061300NH2.nc']

In [84]:
# Read both files
ds1 = xr.open_dataset(tsP_files[0], decode_times=False)
ds2 = xr.open_dataset(tsP_files[1], decode_times=False)
ds3 = xr.open_dataset(tsP_files[2], decode_times=False)

In [85]:
# By printing the xarray dataset, we can inspect the structure of the source data.
ds1

<xarray.Dataset> Size: 5kB
Dimensions:            (row: 43)
Dimensions without coordinates: row
Data variables: (12/14)
    line               (row) |S2 86B ...
    longitude          (row) float64 344B ...
    latitude           (row) float64 344B ...
    station_code       (row) |S10 430B ...
    time               (row) float64 344B ...
    station            (row) |S4 172B ...
    ...                 ...
    temperature        (row) float64 344B ...
    salinity           (row) float64 344B ...
    density            (row) float64 344B ...
    fluorescence       (row) float64 344B ...
    project            (row) |S2 86B ...
    transect           (row) |S20 860B ...
Attributes:
    id:                    010903NH05
    observationDimension:  row

In [86]:
# By inspecting the longitudes, latitudes and time of the observations,
# we know that one source netCDF file only contains a single profile at a time for one station.
print(ds1.longitude.data)
print(ds1.latitude.data)
print(ds1.time.data)

[-124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686 -124.12999686 -124.12999686
 -124.12999686 -124.12999686 -124.12999686]
[44.65169887 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887
 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887
 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887
 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887
 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887 44.65169887
 44.65169887 44.6516988

In [87]:
# Get the longitudes of all stations
lon1 = np.unique(ds1.longitude.data)
lon2 = np.unique(ds2.longitude.data)
lon3 = np.unique(ds3.longitude.data)
lon = np.concatenate([lon1, lon2, lon3])
print("The longitude of each station is: ", lon)

# Get the latitudes of all stations
lat1 = np.unique(ds1.latitude.data)
lat2 = np.unique(ds2.latitude.data)
lat3 = np.unique(ds3.latitude.data)
lat = np.concatenate([lat1, lat2, lat3])
print("The latitude of each station is: ", lat)

# Get the station id of all stations
station1 = np.unique(ds1.station.data)
station2 = np.unique(ds2.station.data)
station3 = np.unique(ds3.station.data)
station = np.concatenate([station1, station2, station3])
print("The station name of each station is: ", station)


The longitude of each station is:  [-124.12999686 -124.12999686 -124.41169686]
The latitude of each station is:  [44.65169887 44.65169887 44.65169887]
The station name of each station is:  [b'NH03' b'NH03' b'NH15']


In [88]:
# Get the IDs of all profiles
profID1 = np.unique(ds1.station_code.data)
profID2 = np.unique(ds2.station_code.data)
profID3 = np.unique(ds3.station_code.data)
profID = np.concatenate([profID1, profID2, profID3])
print("The profile IDs of the two stations are: ", profID)

# Get the number of observations for all profile
rowSize1 = ds1.sizes['row']
rowSize2 = ds2.sizes['row']
rowSize3 = ds3.sizes['row']
rowSize = [rowSize1, rowSize2, rowSize3]
print("The number of observations for each profile is: ", rowSize)

The profile IDs of the two stations are:  [b'010903NH03' b'011207NH03' b'061300NH15a']
The number of observations for each profile is:  [43, 46, 83]


In [ ]:
# time

```{note}
For the convenience of elaboration, we only took two profiles from two stations into account; it's better to include loops in the script. You can find an practice showcase with loop in chapter?
```